In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
data = pd.read_excel('exog.xlsx', parse_dates=['report_dt'])
data = data[['report_dt', 'value']].set_index('report_dt').sort_index().squeeze().asfreq('M')
data.plot()
plt.show()

In [ ]:
data

In [ ]:
train, test = data[:-7], data[-7:]
model = SARIMAX(train, order=(3, 3, 3), seasonal_order=(1, 1, 0, 12), 
                mle_regression=False, enforce_stationarity=False, enforce_invertibility=False).fit()
forecast = model.forecast(len(test))

MAPE = abs(forecast / test - 1).mean()
sMAPE = (f'{MAPE:0.2%}')

plt.suptitle('MAPE: ' + sMAPE)
data.plot()
forecast.append(train.last('M')).sort_index().plot()
plt.show()

In [ ]:
model = SARIMAX(data, order=(3, 3, 3), seasonal_order=(1, 1, 0, 12), 
                mle_regression=False, enforce_stationarity=False, enforce_invertibility=False).fit()
forecast = model.forecast(12)

data.plot()
forecast.append(data.last('M')).sort_index().plot()
plt.show()

In [ ]:
full_forecast = data.append(forecast).sort_index()
full_forecast.to_excel('exog_forecast.xlsx')

In [ ]:
full_forecast

In [ ]:
from itertools import product
params = list(product(range(1, 4), range(1, 4), range(1, 4), range(2), range(2), range(1)))
len(params), params

In [ ]:
MAPE_dict = {}

for param in params:
    model = SARIMAX(train, order=(param[0], param[1], param[2]),
                    seasonal_order=(param[3], param[4], param[5], 12),
                    mle_regression=False, enforce_stationarity=False, enforce_invertibility=False).fit()
    forecast = model.forecast(len(test))
    MAPE = abs(forecast / test - 1).mean()
#     sMAPE = (f'{MAPE:0.2%}')
    MAPE_dict.update({MAPE: param})

In [ ]:
MAPE_dict

In [ ]:
min_MAPE = min(list(MAPE_dict.keys()))
min_MAPE, MAPE_dict[min_MAPE]